In [ ]:
import merged_dataframes
from confluent_kafka import Producer
import json
import pandas as pd



In [ ]:
Bootstrap_server = 'pkc-9q8rv.ap-south-2.aws.confluent.cloud:9092'
API_key = 'XOZNRBMGQYEJNLMX'
API_secret = '99ETgxUsjRP/XYXrMJ9jE3GxOqrsYErw4hRfE3i/VZzcylc6xPivFODrwfi5ovUQ'
SECURITY_PROTOCOL = 'SASL_SSL'
SSL_MACHENISM = 'PLAIN'

producer_config = {
    'sasl.mechanism': SSL_MACHENISM,
    'bootstrap.servers':Bootstrap_server,
    'security.protocol': SECURITY_PROTOCOL,
    'sasl.username': API_key,
    'sasl.password': API_secret
    'enable.idempotence': True   #to ensure indempotence means to neglet duplicate data
}

producer = Producer(producer_config)

In [ ]:
topics = [ 'Top_10_open','Top_10_close','Top_10_volume' ]

In [ ]:
def produce_records(df,topic:str):
    for _, row in df.iterrows():
        # Convert the key and value to JSON format
        # key = json.dumps(row['key'])
        # value = json.dumps(row['value'])
        the_message = row.to_dict()    
        message = json.dumps(the_message)
        # Send record to Kafka topic
        producer.produce(topic,value=message)

    # Flush the producer to ensure all messages are sent
    producer.flush()
    

In [ ]:
last_offset = -1
try:
    with open ('checkpoints.txt','r') as f:
        last_offset = int(f.readline())
except FileNotFoundError:
    pass

In [ ]:
new_records = df[df.index > last_offset]

if not new_records.empty:
    produce_records(new_records)
    
    last_offset = df.index.max()
    
    with open ('checkpoints.txt','r') as f:
        f.write(str(last_offset))
        
        
